
# Problem Set 3
# Due Wednesday, November 13, 2019

<div style="color: #000000;background-color: #FFEEFF">
In this problem set, you are to implement a two-layer (2 layers of weights, 1 hidden layer of units) neural network for binary classification.  All non-linearities are to be sigmoids.

Details are given below.  *Please read the **entire** notebook carefully before proceeding.*

You need to both fill in the necesary code, **and** answer the question at the bottom.</div>

In [81]:
# Below are the only imports that are necessary (or allowed)
import numpy as np
import h5py 
import matplotlib.pyplot as plt
import time
from IPython import display

## Data

<div style="color: #000000;background-color: #FFEEFF">
We will be using a USPS digit dataset (provided in the file uspsall73.mat).
It has 16-by-16 grayscale images of each of the 10 different hand-written digits
However, we will load only two of the digits to use as the two classes in
binary classification
</div>

In [82]:
# function to load two of the 10 classes (c1 is for Y=+1 and c2 is for Y=0)
# Note that for neural networks, we will be using Y={+1,0} instead of Y={+1,-1}
def loaddigitdata(c1,c2,m):
    f = h5py.File('uspsall73.mat','r') 
    data = f.get('data') 
    data = np.array(data).astype(float)
    X = np.concatenate((data[c1,:,:],data[c2,:,:]))
    Y = np.concatenate((np.zeros((data.shape[1])),np.ones((data.shape[1]))))
    
    rndstate = np.random.get_state() # going to set the "random" shuffle random seed
    np.random.seed(132857) # setting seed so that dataset is consistent
    p = np.random.permutation(X.shape[0])
    X = X[p] # this and next line make copies, but that's okay given how small our dataset is
    Y = Y[p]
    np.random.set_state(rndstate) # reset seed
    
    trainX = X[0:m,:] # use the first m (after shuffling) for training
    trainY = Y[0:m,np.newaxis]
    validX = X[m:,:] # use the rest for validation
    validY = Y[m:,np.newaxis]
    return (trainX,trainY,validX,validY)

# In case you care (not necessary for the assignment)
def drawexample(x,ax=None): # takes an x *vector* and draws the image it encodes
    if ax is None:
        plt.imshow(np.reshape(x,(16,16)).T,cmap='gray')
    else:
        ax.imshow(np.reshape(x,(16,16)).T,cmap='gray')

In [83]:
# load the data, to differentiate between 7s and 9s
# we will use on 110 examples for training (50% of the data) and the other half for validation
(trainX,trainY,validX,validY) = loaddigitdata(6,8,1100)
means = trainX.mean(axis=0)
stddevs = trainX.std(axis=0)
stddevs[stddevs<1e-6] = 1.0
trainX = (trainX-means)/stddevs
validX = (validX-means)/stddevs


# Convert this cell to a code cell if you wish to see each of the examples, plotted
# (completely not necessary for the problem set)
f = plt.figure()
f.set_size_inches(8,8)

ax = f.add_subplot(111)
plt.ion()
f.canvas.draw()
for exi in range(trainX.shape[0]):
    display.clear_output(wait=True)
    drawexample(trainX[exi,:])
    digitid = (9 if trainY[exi]>0 else 7)
    ax.set_title('y = '+str(int(trainY[exi]))+" ["+str(digitid)+"]")
    display.display(f)
    #time.sleep(0.1)

## WRITE `nneval` and `trainneuralnet` [20 points]

<div style="color: #000000;background-color: #FFFFEE">
This is the main portion of the assignment

Note that the $Y$ values are +1 and 0 (not +1 and -1).  This is as in class and works better with a sigmoid output.

You need to write the two functions below (plus any more you would like to add to help): `nneval` and `trainneuralnet`.  The first takes an array/matrix of X vectors and the weights from a neural network and returns a vector of predicted Y values (should be numbers between 0 and 1 -- the probability of class +1, for each of the examples).  The second takes a data set (Xs and Ys), the number of hidden units, and the lambda value (for regularization), and returns the weights.  W1 are the weights from the input to the hidden and W2 are the weights from the hidden to the output.

A few notes:
- **Starting Weights**: The code supplied randomly selects the weights near zero.  https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79 has a reasonable explanation of why we are doing it this way.  But for the purposes of the assignment, you can just accept this is a good way to initialize neural network weights.
- **Offset Terms**: Each layer should have an "offset" or "intercept" unit (to supply a 1 to the next layer), except the output layer.
- **Batch Updates**: For a problem this small, use batch updates.  That is, the step is based on the sum of the gradients for each data point in the training set.
- **Step Size**: http://ruder.io/optimizing-gradient-descent/index.html#gradientdescentoptimizationalgorithms describes a number of methods to adaptively control $\eta$ for fast convergence.  You don't need to understand any of them; however, without them, convergence to good solutions on this problem can be quite slow.  Therefore, *use RMSprop*: the code below has a simple version of RMSprop that is sufficient for this assignment.  You need to supply the code that calculates `sumofgrad2` which should be the sum of the square of each element of the gradient (the squared length of the gradient).  (for debugging, feel free to use a constant $\eta$). 
- **Stopping Criterion**: To determine when to stop, check the loss function every 10 iterations.  If it has not improved by at least $10^{-8}$ over those 10 iterations, stop.
- **Regularization**: You should penalize (from the regularization) all of the weights, even those coming out of offset units.  While it makes sense sometimes not to penalize the ones for the constant $1$ units, you'll find this easier if you just penalize them all.

Tips that might help:
- Display the loss function's value every 10 iterations (or so).  It should generally be getting smaller.
- The smaller $\lambda$ is and the more units, the more difficult (long) the optimization will be.
- Write a function to do forward propagation and one to do backward propagation.  Write a function to evaluate the loss function.  In general, break things up to keep things straight.
- Processing the entire batch at once is more efficient in numpy.  Use numpy broadcasting to avoid loops where possible.
</div>

In [86]:
### FEEL FREE TO ADD HELPER FUNCTIONS
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def loss(f, Y):
    if f == 0:
        return -((1 - Y) * np.log(1 - f))
    return -(Y * np.log(f) + (1 - Y) * np.log(1 - f))

def addones(z):
    if len(z.shape) == 1:
        return np.concatenate((np.ones(1), z))
    else:
        m = z.shape[0]
        return np.hstack((np.ones((m,1)), z))
    
def gprime(z):
    return sigmoid(z) * (1 - sigmoid(z))

def nneval(X,W1,W2):
    # W1 and W2 are as from trainneuralnet
    # X is number of examples by number of features
    # this should return a vector that has length of the number of examples
    # the values in the return vector should be the predicted probabilities of class +1
    (m,n) = X.shape
    X = addones(X)
    Y = np.zeros(m)
    i = 0
    for i in range(m):
        Z1 = W1@X[i]
        A = addones(sigmoid(Z1))
        Z2 = W2@A
        Y[i] = sigmoid(Z2)
    return Y
  
def trainneuralnet(X,Y,nhid,lam):
    (m,n) = X.shape
    W1 = (np.random.rand(nhid,n+1)*2-1)*np.sqrt(6.0/(n+nhid+1)) # weights to each hidden unit from the inputs (plus the added offset unit)
    W2 = (np.random.rand(1,nhid+1)*2-1)*np.sqrt(6.0/(nhid+2)) # weights to the single output unit from the hidden units (plus the offset unit)
    W1[:,0] = 0
    W2[:,0] = -nhid/2.0
    (W1_m,W1_n)= W1.shape
    Eg2=1
    check = 0
    cnt = 0
    deltaF = np.zeros(m)
    deltaH = np.zeros((m, nhid))
    gradF = np.zeros((m, nhid+1))
    gradH = np.zeros((m, W1_m, W1_n))
    totH = W1
    totH = totH - W1
    totF = W2
    totF = totF - W2
    saveL = 0
    totL = 0
    while 1:
        for t in range(m):
            Z1 = W1 @ addones(X[t])
            A = addones(sigmoid(Z1))
            Z2 = W2 @ A
            f = sigmoid(Z2)
            deltaF[t] = f - Y[t]
            deltaH[t] = gprime(Z1) * (W2[0,1:].T * deltaF[t])
            gradF[t] = deltaF[t] * A    
            gradH[t] = deltaH[t][:, None] @ addones(X[t])[None,:]
            eta = 0.001
            totH = totH + (gradH[t]) + (2 * lam * W1 / m)
            totF = totF + (gradF[t]) + (2 * lam * W2 / m)
            totL = totL + loss(f, Y[t])
        sumofgrad2 = (gradH * gradH).sum() + (gradF * gradF).sum()#??? sum of the squares of all of the elements of the gradient
        Eg2 = 0.9*Eg2 + 0.1*sumofgrad2
        eta = 0.1/(np.sqrt((1e-10+Eg2)))
        W1 = W1 - (eta * totH)
        W2 = W2 - (eta * totF)
        cnt = cnt + 1
        L = (totL / m) + (lam * ((W1 * W1).sum() + (W2 * W2).sum()) / m)
        if cnt == 10:
            cnt = 0
            print(saveL)
            print(np.fabs(saveL - L))
            if np.fabs(saveL - L) < 1e-8:
                break
            saveL = L
        totL = 0
        totH = 0
        totF = 0
    # use eta here to make update
    return (W1,W2)

In [87]:
# Use this cell (or others you add) to check your network
# I would debug on simple examples you create yourself (trying to understand what happens with
#  the full 256-dimensional data is hard)
   
(W1,W2) = trainneuralnet(trainX,trainY,5,1) #an example of training on the USPS data with 5 hidden units and lambda=1
print(W1)
print(W2)

0
[0.11214475]
[0.11214475]
[0.03159817]
[0.08054658]
[0.00997002]
[0.07057656]
[0.00519485]
[0.06538171]
[0.00363558]
[0.06174613]
[0.00284483]
[0.0589013]
[0.00182262]
[0.05707867]
[0.00139869]
[0.05567999]
[0.00111557]
[0.05456442]
[0.00090087]
[0.05366355]
[0.0007256]
[0.05293794]
[0.00058285]
[0.05235509]
[0.00046882]
[0.05188627]
[0.00037908]
[0.0515072]
[0.00031082]
[0.05119638]
[0.00026356]
[0.05093281]
[0.00023723]
[0.05069558]
[0.00023209]
[0.05046349]
[0.00025302]
[0.05021047]
[0.00024908]
[0.04996139]
[0.0001642]
[0.04979719]
[0.00011523]
[0.04968197]
[8.69114648e-05]
[0.04959505]
[6.89365551e-05]
[0.04952612]
[5.65524452e-05]
[0.04946957]
[4.74489361e-05]
[0.04942212]
[4.04389665e-05]
[0.04938168]
[3.48652809e-05]
[0.04934681]
[3.03343821e-05]
[0.04931648]
[2.6592628e-05]
[0.04928989]
[2.3465116e-05]
[0.04926642]
[2.08237794e-05]
[0.0492456]
[1.85700077e-05]
[0.04922703]
[1.66249582e-05]
[0.0492104]
[1.49242523e-05]
[0.04919548]
[1.34153713e-05]
[0.04918206]
[1.20566798e-0

## Performance plot
<div style="color: #000000;background-color: #FFFFEE">
The code below will plot your algorithm's error rate on this data set for various regularization strengths and numbers of hidden units.

Make sure your code works for this plot.

My code runs in about 5 minutes (to produce the full plot below)
</div>

In [88]:
# solutions from PS 2's logistic regression (modified to work with Y=0/1)
# for comparison... see plot below
def learnlogreg(X,Y,lam):    
    (m,n) = X.shape
    w = np.zeros((n+1))
    
    eta = 0.05
    
    Y11 = Y.copy()
    Y11[Y11<0.5] = -1
    XY = Y11*np.hstack((np.ones((m,1)),X))
    neggrad = np.ones(n)
    while(neggrad.dot(neggrad)>1e-5):
        predy = sigmoid(XY@w)[:,np.newaxis]
        neggrad = ((1.0-predy)*XY).mean(axis=0)
        gradreg = (lam/m)*w*2.0
        gradreg[0] = 0
        neggrad = neggrad-gradreg
        
        llh = -np.sum(np.log(predy))/m + lam*(np.sum(w*w)-w[0]*w[0])/m
        
        w = w + eta*neggrad
    
    return w
    
def predictlogreg(X,w):
    return sigmoid(np.hstack((np.ones((X.shape[0],1)),X))@w)

In [ ]:
def setupfig():
    f = plt.figure()
    f.set_size_inches(8,8)
    ax = f.add_subplot(111)
    plt.ion()
    f.canvas.draw()
    return (f,ax)

def plotit(lams,nhiddens,erates,f,ax):
    ax.clear()
    for i in range(nhiddens.shape[0]):
        ax.plot(lams,erates[:,i],'*-')
    ax.set_yscale('log',subsy=[1,2,3,4,5,6,7,8,9])
    ax.set_yticks([0.1,0.01])
    ax.set_xscale('log')
    f.canvas.draw()
    ax.set_xlabel('lambda')
    ax.set_ylabel('validation error rate')
    ax.legend([(('# hidden units = '+str(x)) if x>0 else 'logistic regression') for x in nhiddens])
    display.display(f)
    display.clear_output(wait=True)
    
    
nhiddens = np.array([0,4,20,100])
lams = np.logspace(-3,2,10)
erates = np.empty([lams.shape[0],nhiddens.shape[0]])
erates[:,:] = np.nan

(f,ax) = setupfig()

    
for ni, nhid in enumerate(nhiddens):
    for li, lam in reversed(list(enumerate(lams))):
        if ni==0:
            w = learnlogreg(trainX,trainY,lam)
            predy = predictlogreg(validX,w)[:,np.newaxis]
        else:
            (W1,W2) = trainneuralnet(trainX,trainY,nhid,lam)
            predy = nneval(validX,W1,W2)
        predy[predy<0.5] = 0.0
        predy[predy>=0.5] = 1.0
        validerr = (predy != validY).mean()
        erates[li,ni] = validerr
        
        plotit(lams,nhiddens,erates,f,ax)

0
[0.0905018]
[0.0905018]
[0.0351028]
[0.05539901]
[0.0141844]
[0.0412146]
[0.00704944]
[0.03416517]
[0.00414276]
[0.03002241]
[0.00264872]
[0.02737369]
[0.00181633]
[0.02555736]
[0.00147414]
[0.02408322]
[0.00327281]
[0.02081042]
[0.00190256]
[0.01890785]
[0.0008321]
[0.01807575]
[0.0004473]
[0.01762846]
[0.00030058]
[0.01732787]
[0.00021966]
[0.01710821]
[0.00017017]
[0.01693804]
[0.00013826]
[0.01679978]
[0.00011633]
[0.01668344]
[0.00010083]
[0.01658262]
[9.6396777e-05]
[0.01648622]
[0.00118159]
[0.01530463]
[0.00016719]
[0.01513743]
[3.03347945e-05]
[0.0151071]
[0.00013732]
[0.01496978]
[3.23385304e-05]
[0.01500212]
[1.78433673e-05]
[0.01498427]
[0.00021354]
[0.01477074]
[9.60215317e-05]
[0.01486676]
[7.48224275e-05]
[0.01479194]
[6.45805841e-05]
[0.01472736]
[3.27664603e-05]
[0.01469459]
[2.52375967e-05]
[0.01466935]
[2.0559109e-05]
[0.01464879]
[1.70642835e-05]
[0.01463173]
[1.44058249e-05]
[0.01461732]
[1.23518161e-05]
[0.01460497]
[1.07746975e-05]
[0.0145942]
[9.65058129e-06]


## INTERPRET the Plot [5 points]
<div style="color: #000000;background-color: #FFFFEE">
How do you interpret the plot above?  How and why does the plot differ by number of hidden units?  By $\lambda$ value?  What parts of this plot agree with the material taught?  What parts do not?
</div>

### Your Answer Here